# Evaluation of Candidate Models

## 1. Imports and Loading Data

In [11]:
#0. Imports and config
#update system path
import warnings
warnings.filterwarnings('ignore')
import os
import sys
wd = globals()['_dh'][0]

#imports
#from statsmodels.tsa.stattools import adfuller
import pandas as pd
import numpy as np
import configparser
import ast
from functions.evaluation_functions import *
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error #evaluation metric used to construct RMSE
from joblib import dump, load
import time
import matplotlib.pyplot as plt
#config
config = configparser.ConfigParser()
config.read('config.ini')

['config.ini']

In [12]:
# 1. Load Data
t0 = time.time()
partial_df = pd.read_csv(f"{wd}{config['data']['partial_data_output_path']}")
full_df = pd.read_csv(f"{wd}{config['data']['full_data_output_path']}")
optimised_models_list = load(f"{wd}/{config['model_prep']['optimised_models_list_path']}")
data = load(f"{wd}/{config['model_prep']['scaled_data_path']}")
t1 = time.time()
print("Data loading took", (t1 - t0), "seconds")

Data loading took 0.0660254955291748 seconds


## 2. Summary Evaluation Measures for each model

In [13]:
#2. Produce summary stats     
t0 = time.time() 
summary_df = evaluate_models(data, optimised_models_list)
t1 = time.time()
print("Summary evaluation measures took", (t1 - t0), "seconds")
summary_df.sort_values(by = 'RMSE')

Summary evaluation measures took 0.051854610443115234 seconds


,Model_details,RMSE,MAE,R2
2,XGBoost_full,5.117775,1.956268,-0.060784
0,Random Forest_full,5.173952,1.946382,-0.084200
3,Random Forest_partial,5.870205,2.437368,-0.047604
5,XGBoost_partial,5.871881,2.485527,-0.048202
1,ARDR_full,7.013704,3.259338,-0.992322
4,ARDR_partial,7.144875,4.347361,-0.551958


## 3. View predictions different models make, including a naive model using past values

In [14]:
#3. View model predictions
t0 = time.time() 
eval_df = make_predictions_df(partial_df, full_df, optimised_models_list, horizon = ast.literal_eval(config['model_prep']['horizon']))
eval_df['Naive AR model'] = eval_df['gdp_growth'].shift(ast.literal_eval(config['model_prep']['horizon']) -1) # A Naive lag model
predictions_df = eval_df.iloc[:, -(len(optimised_models_list) +2):]
predictions_df['date'] = eval_df['date']
t1 = time.time()
print("Making Estimations took", (t1 - t0), "seconds")

Making Estimations took 0.0630040168762207 seconds


## 4. Interactive plot of results

In [15]:
#4. Plot Time Chart
plt.rcParams["figure.figsize"] = (20, 10)
#import plotly.io as pio
#pio.renderers.default='browser'
start_date = config['evaluation']['start_date']
end_date = config['evaluation']['end_date']
palette = ast.literal_eval(config['evaluation']['boe_palette'])

plot_interactive_chart(predictions_df, start_date, end_date, palette)